#### Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Load data from Wikipedia

In [2]:
# Get 
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# Beautify
s = BeautifulSoup(r.content,"lxml")

# Get raw text 
table = s.find_all('table')[0]

#### Transform to dataframe

In [3]:
# Create dictionary which will be used to construct dataframe
post_code = []
borough = []
neighborhood = []

for row in table.find_all('tr'):
    i = 0
    for col in row.find_all('td'):
        if i == 0:
            post_code.append(col.get_text())
        elif i == 1:
            borough.append(col.get_text())
        else:
            neighborhood.append(col.get_text()[:-1])
        i += 1

d = {'PostalCode': post_code, 'Borough': borough, 'Neighborhood': neighborhood}

In [4]:
# Create Dataframe
df = pd.DataFrame(d)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Remove records with unassigned borough

In [5]:
# Remove records with unassigned borough
df = df[df['Borough'] != 'Not assigned']

In [6]:
# Reindex
df = df.reset_index(drop = True)

#### Assign neighborhood as borough's name

In [7]:
def get_borough(frame):
    if frame['Neighborhood'] == 'Not assigned':
        return frame['Borough']
    else:
        return frame['Neighborhood']
    
df['Neighborhood'] = df.apply(get_borough, axis = 1)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Combine neighborhood

In [8]:
# Combine boroughs
dfx = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
dfx.head()

,PostalCode,Neighborhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [9]:
# Merge with original dataframe
dfx = dfx.merge(df.drop('Neighborhood', axis=1), on = 'PostalCode', how = 'left')
dfx.head()

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge,Malvern",Scarborough
1,M1B,"Rouge,Malvern",Scarborough
2,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
3,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
4,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough


In [10]:
# Reorder clumns
dfx = dfx[['PostalCode','Borough','Neighborhood']]
dfx.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
4,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"


#### Load coordinates of postal codes

In [13]:
location = pd.read_csv('http://cocl.us/Geospatial_data')
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join to current dataframe

In [15]:
dfy = dfx.merge(location, left_on = 'PostalCode', right_on = 'Postal Code', how = 'left').\
            drop('Postal Code', axis = 1)
dfy.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
4,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
